# Setup

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
import random
import time
import torch
import torch.nn as nn
import torch.optim
import torchvision.datasets as datasets
import torchvision.models as models
import torchvision.transforms as transforms
import wandb
import warnings
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, classification_report, confusion_matrix, ConfusionMatrixDisplay
from torch.utils.data import DataLoader
from torchvision.transforms.functional import InterpolationMode
from torch.nn.parallel import DataParallel

device = torch.device("cuda")
if torch.cuda.device_count() >= 1:
    print("Using", torch.cuda.device_count(), "GPUs!")
    parallel = True
else:
    parallel = False

warnings.filterwarnings("ignore")
random.seed(7)
np.random.seed(7)
torch.manual_seed(7)

Using 2 GPUs!


In [ ]:
# Define Parameters
batch_size = 128
epochs = 150
lr = 1e-5

# Other Parameters
class_label_map = {"severe": 2, "mild": 1, "little_or_none": 0}
# class_label_map = {"landslide":6, "other_disaster":5, "fire":4, "hurricane":3, "earthquake":2, "flood":1, "not_disaster":0}
# class_label_map = {"affected_injured_or_dead_people": 3, "rescue_volunteering_or_donation_effort": 2, "infrastructure_and_utility_damage": 1, "not_humanitarian": 0}
# class_label_map = {"informative": 1, "not_informative": 0}

num_classes = len(class_label_map)
task = 'Damage_Severity_Classification' 
# task = 'Disaster_Types_Classification'
# task = 'Humanitarian_Classification'
# task = 'Informativeness_Classification'

weights_denet121 = models.DenseNet121_Weights.IMAGENET1K_V1
weights_effnetb1 = models.EfficientNet_B1_Weights.IMAGENET1K_V1
weights_mobnetv2 = models.MobileNet_V2_Weights.IMAGENET1K_V1
weights_resnet18 = models.ResNet18_Weights.IMAGENET1K_V1
weights_resnet50 = models.ResNet50_Weights.IMAGENET1K_V1
weights_resnet101 = models.ResNet101_Weights.IMAGENET1K_V1
weights_vgg16 = models.VGG16_Weights.IMAGENET1K_V1

wandb.login()
wandb.init(project='Reproduce_exp', 
           config={
               "learning_rate": lr,
               "epochs": epochs,
               "batch_size": batch_size,
           },
           name='reproduce')

# Data Loading

In [4]:
tr_normalize = transforms.Normalize(
    mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])

train_dataset = datasets.ImageFolder(os.path.join('/work/u9562361/crisis_vision_benchmarks/Damage Severity', "train"), 
                                    transform = transforms.Compose([
                                        transforms.Resize((224, 224), interpolation=InterpolationMode.BICUBIC),
                                        transforms.ToTensor(),
                                        tr_normalize,
                                    ]))

test_dataset = datasets.ImageFolder(os.path.join('/work/u9562361/crisis_vision_benchmarks/Damage Severity', "val"))
dev_dataset = datasets.ImageFolder(os.path.join('/work/u9562361/crisis_vision_benchmarks/Damage Severity', "dev"))

test_dataset.transform = transforms.Compose([
    transforms.Resize((224, 224), interpolation=InterpolationMode.BICUBIC),
    transforms.ToTensor(),
    tr_normalize,
])

dev_dataset.transform = transforms.Compose([
    transforms.Resize((224, 224), interpolation=InterpolationMode.BICUBIC),
    transforms.ToTensor(),
    tr_normalize,
])

trainloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
testloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=4)
devloader = DataLoader(dev_dataset, batch_size=batch_size, shuffle=True, num_workers=4)

# Model

In [5]:
# model = models.densenet121(weights=weights_denet121)
# model = models.efficientnet_b1(weights=weights_effnetb1)
model = models.mobilenet_v2(weights=weights_mobnetv2)
# model = models.resnet18(weights=weights_resnet18)
# model = models.resnet50(weights=weights_resnet50)
# model = models.resnet101(weights=weights_resnet101)
# model = models.vgg16(weights=weights_vgg16)

# DenseNet Architecture
# num_ftrs = model.classifier.in_features
# model.classifier = nn.Linear(num_ftrs, num_classes)

# EfficientNet, MobileNet, VGG Architecture
num_ftrs = model.classifier[-1].in_features
model.classifier[-1] = nn.Linear(num_ftrs, num_classes)

# ResNet Architecture
# num_ftrs = model.fc.in_features
# model.fc = nn.Linear(num_ftrs, num_classes)

model = DataParallel(model)
model = model.to(device)

# Parameters
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.1, patience=10, mode='max')
criterion = nn.CrossEntropyLoss()

train_logs = {"loss": [], "accuracy": [], "precision": [], "recall": [], "f1-score": [], "time": []}
val_logs = {"loss": [], "accuracy": [], "precision": [], "recall": [], "f1-score": [], "time": []}

In [6]:
# Learning schedule update
def dev_one_epoch(dev_data_loader):
    epoch_loss = []
    epoch_acc = []

    model.eval()

    with torch.no_grad():
        for images, labels in dev_data_loader:
            images = images.to(device)
            labels = labels.to(device)

            preds = model(images)  # Forward

            # Calculating Loss
            loss = criterion(preds, labels)
            epoch_loss.append(loss.item())

            # Calculating Accuracy
            _, predicts = preds.max(1)
            predicts = predicts.detach().cpu().numpy()
            labels = labels.detach().cpu().numpy()
            acc = accuracy_score(labels, predicts)
            epoch_acc.append(acc)

    # Acc and Loss
    epoch_loss = np.mean(epoch_loss)
    epoch_acc = np.mean(epoch_acc)

    return epoch_loss, epoch_acc

In [7]:
def train_one_epoch(train_data_loader):
    epoch_loss = []
    epoch_acc = []
    epoch_ps = []
    epoch_rs = []
    epoch_f1 = []
    trues = []
    prediction = []
    start_time = time.time()

    model.train()

    for batch_idx, (images, labels) in enumerate (train_data_loader):
        images = images.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        preds = model(images)
        loss = criterion(preds, labels)

        # Calculating Loss
        epoch_loss.append(loss.item())

        # Calculating Metrics
        _, predicts = preds.max(1)
        predicts = predicts.detach().cpu().numpy()
        labels = labels.detach().cpu().numpy()
        acc = accuracy_score(labels, predicts)
        ps = precision_score(labels, predicts, average="weighted")
        rs = recall_score(labels, predicts, average="weighted")
        f1 = f1_score(labels, predicts, average="weighted")
        
        epoch_acc.append(acc)
        epoch_ps.append(ps)
        epoch_rs.append(rs)
        epoch_f1.append(f1)
        trues.append(labels)
        prediction.append(predicts)

        # Backpropagation
        loss.backward()
        optimizer.step()

    trues = np.concatenate(trues)
    prediction = np.concatenate(prediction)
    accuracy = accuracy_score(trues, prediction)
    precision = precision_score(trues, prediction, average="weighted")
    recall = recall_score(trues, prediction, average="weighted")
    f1score = f1_score(trues, prediction, average="weighted")
    accuracy = accuracy * 100
    precision = precision * 100
    recall = recall * 100
    f1score = f1score * 100

    # Overall Epoch Results
    end_time = time.time()
    total_time = end_time - start_time

    epoch_loss = np.mean(epoch_loss)
    epoch_acc = np.mean(epoch_acc) * 100
    epoch_ps = np.mean(epoch_ps) * 100
    epoch_rs = np.mean(epoch_rs) * 100
    epoch_f1 = np.mean(epoch_f1) * 100

    train_logs["loss"].append(epoch_loss)
    train_logs["accuracy"].append(accuracy)
    train_logs["precision"].append(precision)
    train_logs["recall"].append(recall)
    train_logs["f1-score"].append(f1score)
    train_logs["time"].append(total_time)
    wandb.log({"train_loss": epoch_loss, "train_f1": f1score})

    return epoch_loss, accuracy, precision, recall, f1score, total_time

def val_one_epoch(val_data_loader, best_val_f1):
    epoch_loss = []
    epoch_acc = []
    epoch_ps = []
    epoch_rs = []
    epoch_f1 = []
    trues = []
    prediction = []
    start_time = time.time()

    model.eval()

    with torch.no_grad():
        for images, labels in val_data_loader:
            images = images.to(device)
            labels = labels.to(device)

            preds = model(images)

            # Calculating Loss
            loss = criterion(preds, labels)
            epoch_loss.append(loss.item())

            # Calculating Metrics
            _, predicts = preds.max(1)
            predicts = predicts.detach().cpu().numpy()
            labels = labels.detach().cpu().numpy()
            acc = accuracy_score(labels, predicts)
            ps = precision_score(labels, predicts, average="weighted")
            rs = recall_score(labels, predicts, average="weighted")
            f1 = f1_score(labels, predicts, average="weighted")
            epoch_acc.append(acc)
            epoch_ps.append(ps)
            epoch_rs.append(rs)
            epoch_f1.append(f1)
            trues.append(labels)
            prediction.append(predicts)
    
    trues = np.concatenate(trues)
    prediction = np.concatenate(prediction)
    accuracy = accuracy_score(trues, prediction)
    precision = precision_score(trues, prediction, average="weighted")
    recall = recall_score(trues, prediction, average="weighted")
    f1score = f1_score(trues, prediction, average="weighted")
    accuracy = accuracy * 100
    precision = precision * 100
    recall = recall * 100
    f1score = f1score * 100
    cm = confusion_matrix(trues, prediction)
    cr = classification_report(trues, prediction)

    # Overall Epoch Results
    end_time = time.time()
    total_time = end_time - start_time

    epoch_loss = np.mean(epoch_loss)
    epoch_acc = np.mean(epoch_acc) * 100
    epoch_ps = np.mean(epoch_ps) * 100
    epoch_rs = np.mean(epoch_rs) * 100
    epoch_f1 = np.mean(epoch_f1) * 100

    val_logs["loss"].append(epoch_loss)
    val_logs["accuracy"].append(accuracy)
    val_logs["precision"].append(precision)
    val_logs["recall"].append(recall)
    val_logs["f1-score"].append(f1score)
    val_logs["time"].append(total_time)
    wandb.log({"val_loss": epoch_loss, "val_f1": f1score})

    # Saving best model
    if f1score > best_val_f1:
        best_val_f1 = f1score
        torch.save(model.state_dict(), "reproduce.pth")

    return epoch_loss, accuracy, precision, recall, f1score, total_time, best_val_f1, cm, cr

# Results

In [ ]:
best_val_acc = 0
best_val_ps = 0
best_val_rs = 0
best_val_f1 = 0

output_list = [] 

for epoch in range(epochs):
    train_loss, train_acc, train_ps, train_rs, train_f1, train_time = train_one_epoch(trainloader) 
    dev_loss, dev_acc = dev_one_epoch(devloader) 
    val_loss, val_acc, val_ps, val_rs, val_f1, val_time, best_val_f1, cm, cr = val_one_epoch(testloader, best_val_f1) 
    
    lr_scheduler.step(dev_acc)

    if val_ps > best_val_ps:
        best_val_ps = val_ps

    if val_rs > best_val_rs:
        best_val_rs = val_rs

    if val_acc > best_val_acc:
        best_val_acc = val_acc

    total_time = train_time + val_time
    output_str = f"Epoch {epoch+1}/{epochs} - loss: {train_loss:.4f} - f1-score: {train_f1:.2f}% - val_loss: {val_loss:.4f} - val_f1-score: {val_f1:.2f}% - time: {total_time:.2f}s"
    output_list.append(output_str)
    print(output_str)

print()
print(task + ' Performance:')
print(f'Accuracy : {best_val_acc:.2f}%')
print(f'Precision : {best_val_ps:.2f}%')
print(f'Recall : {best_val_rs:.2f}%')
print(f'F1-Score : {best_val_f1:.2f}%')
print()
print(task + ' Classification Report:')
print(cr)

disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=None)
disp.plot()
plt.savefig("reproduce-CM.png") 
plt.close() 

wandb.finish()